<a href="https://colab.research.google.com/github/IshaqKHATTAK/Natural-Language-Processing/blob/main/Naive_Bias_2020313.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**naive bayes from scrach using BOW**
1. text preprocessin(SW removal, punctuation removal etc)

In [4]:
df = pd.read_csv("spam_ham_dataset.csv")

In [5]:

df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [6]:
new_df = df[['label', 'text']]
#drop out all the other columns as they are not important

In [7]:
new_df.head()

,label,text
0,ham,Subject: enron methanol ; meter # : 988291\r\n...
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,spam,"Subject: photoshop , windows , office . cheap ..."
4,ham,Subject: re : indian springs\r\nthis deal is t...


In [8]:
# Define a function to preprocess text
def preprocess_text(text):
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Tokenize the text into words
    words = word_tokenize(text.lower())

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if w not in stop_words]

    # Perform stemming using Porter stemmer
    stemmer = PorterStemmer()
    words = [stemmer.stem(w) for w in words]

    # Join the processed words back into a string
    processed_text = ' '.join(words)

    return processed_text

In [9]:
new_df = new_df.assign(processed_text=new_df['text'].apply(preprocess_text))


In [10]:
new_df.head()

,label,text,processed_text
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,subject enron methanol meter 988291 follow not...
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",subject hpl nom januari 9 2001 see attach file...
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",subject neon retreat ho ho ho around wonder ti...
3,spam,"Subject: photoshop , windows , office . cheap ...",subject photoshop window offic cheap main tren...
4,ham,Subject: re : indian springs\r\nthis deal is t...,subject indian spring deal book teco pvr reven...


In [11]:
df_all = new_df[['label','processed_text']]

In [12]:
df_all.head()

,label,processed_text
0,ham,subject enron methanol meter 988291 follow not...
1,ham,subject hpl nom januari 9 2001 see attach file...
2,ham,subject neon retreat ho ho ho around wonder ti...
3,spam,subject photoshop window offic cheap main tren...
4,ham,subject indian spring deal book teco pvr reven...


In [13]:
# Get the number of rows in the DataFrame
num_rows = new_df.shape[0]

# Calculate the number of rows for the training set (80%)
train_size = int(num_rows * 0.8)

# Slice the DataFrame to create the training set
train_df = new_df[:train_size]

# Slice the DataFrame to create the testing set
test_df = new_df[train_size:]

# Print the shape of each set
print(f"Training set shape: {train_df.shape}")
print(f"Testing set shape: {test_df.shape}")
print(f'traning size: {train_size}')

Training set shape: (4136, 3)
Testing set shape: (1035, 3)
traning size: 4136


###vocabulary of all the words

In [14]:
# Create a vocabulary of all unique words in the training set
vocab = set()
for text in train_df["processed_text"]:
    for word in text.split():
        vocab.add(word)

In [15]:
# Compute the probability of each word occurring in spam and ham messages in the training set
spam_words = {} #contain the probability of the words in spam messeges.
ham_words = {}
spam_count = 0 
ham_count = 0
for i in range(len(train_df)):
    row = train_df.iloc[i]
    text = row["processed_text"]
    label = row["label"]
    words = text.split()
    if label == "spam":
        spam_count += 1
        for word in words:
            if word not in spam_words: #palce the new word
                spam_words[word] = 1
            else:
                spam_words[word] += 1 #increase the frequency becoz the word already there
    else:
        ham_count += 1
        for word in words:
            if word not in ham_words:
                ham_words[word] = 1
            else:
                ham_words[word] += 1

In [16]:
print(spam_words['photoshop']) #contain the frequency of each word in spam messges
print(spam_words['window'])
print(spam_words['offic'])

66
171
190


In [17]:
#ham_words

In [18]:
# Compute the PRIOR probabilities of spam and ham messages in the training set
p_spam = spam_count / len(train_df)
p_ham = ham_count / len(train_df)

In [19]:
p_spam

0.28505802707930367

In [20]:
p_ham

0.7149419729206963

In [21]:
# Calculate the likelihood probabilities for each word in the vocabulary
spam_probs = {}
ham_probs = {}
for word in vocab:
    if word in spam_words:
      # p(w_i|c) = count(w_i,c)+1 / count(w,c) + vocab
        spam_probs[word] = (spam_words[word] + 1) / (spam_count + len(vocab))
    else:
        spam_probs[word] = 1 / (spam_count + len(vocab))
    if word in ham_words:
        ham_probs[word] = (ham_words[word] + 1) / (ham_count + len(vocab))
    else:
        ham_probs[word] = 1 / (ham_count + len(vocab))

In [22]:
print(spam_probs['window'])
print(spam_probs['offic'])

0.004393358876117497
0.004878671775223499


In [23]:
#ham_probs

In [24]:
def predict(X):
    """
    Predict the class labels of the given test data X.
    X: a list of strings, where each string represents a document
    """
    y_pred = []
    for doc in X:
        words = doc.split()
        scores = {}
        scores['spam'] = p_spam
        scores['ham'] = p_ham
        for word in words:
            if word in vocab:
                scores['spam'] += np.log(spam_probs[word])
                scores['ham'] += np.log(ham_probs[word])
        y_pred.append(max(scores, key=scores.get))
    return y_pred

In [25]:
predict(['congrats you have won 1 billion dollar'])

['spam']

In [26]:
predict(['congrats you got a job'])

['ham']

In [27]:
test_df = test_df[['text','label']]
test_df.head()

,text,label
4136,Subject: 3 - urgent - to avoid loss of informa...,ham
4137,Subject: nominations for eastrans reciept for ...,ham
4138,Subject: change in operating companies\r\nplea...,ham
4139,Subject: we deliver to your door within 24 hou...,spam
4140,Subject: natural gas nomination for 03 / 01\r\...,ham


In [28]:
y_pred = []
for tweet in test_df['text']:
  y_pred.append(predict([tweet]))


In [29]:
test_df['y_pred'] = y_pred
test_df.head()

,text,label,y_pred
4136,Subject: 3 - urgent - to avoid loss of informa...,ham,[ham]
4137,Subject: nominations for eastrans reciept for ...,ham,[ham]
4138,Subject: change in operating companies\r\nplea...,ham,[ham]
4139,Subject: we deliver to your door within 24 hou...,spam,[ham]
4140,Subject: natural gas nomination for 03 / 01\r\...,ham,[ham]


In [30]:
def remove_brack(df):
  for row in range(len(df)):
    new_string = re.sub(r'[()\[\]{}]', '', df.iloc[i]['y_pred'])

###accuracy

In [31]:
correct =0
for i in range(len(test_df)-1):
  total = len(test_df)
  for word in y_pred[i]:
    if test_df.iloc[i]['label'] == word:
      correct += 1
correct = correct/total  
print(f'accuracy is {correct}')

accuracy is 0.9053140096618357


In [ ]:
#1. Featurization (TF-IDF) + Feature Engineering + ML Model pipeline

#Sklearn naive bayes

**sklearn pipline for naivie bayes**
1. EDA + feature engineering (preprocess and clean the text)
2. sklearn CountVectorizer or tfidf (convert the documents into the numbers)
2. naive bayes classifier type (apply the model to tokens)
3. make pipeline of above (optional)

###preprocessing and cleaning

In [39]:
df['label'] = df['label'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Unnamed: 0,label,text,label_num
0,605,0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,0,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,0,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,1,"Subject: photoshop , windows , office . cheap ...",1
4,2030,0,Subject: re : indian springs\r\nthis deal is t...,0


In [40]:
new_df = df[['text','label']]
new_df.head()

,text,label
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\r\nthis deal is t...,0


In [41]:
new_df = new_df.assign(processed_text=new_df['text'].apply(preprocess_text))
new_df.head()

,text,label,processed_text
0,Subject: enron methanol ; meter # : 988291\r\n...,0,subject enron methanol meter 988291 follow not...
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0,subject hpl nom januari 9 2001 see attach file...
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0,subject neon retreat ho ho ho around wonder ti...
3,"Subject: photoshop , windows , office . cheap ...",1,subject photoshop window offic cheap main tren...
4,Subject: re : indian springs\r\nthis deal is t...,0,subject indian spring deal book teco pvr reven...


In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_df.text,new_df.label)

###countvectorizer

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()[:2]

array([[0, 2, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

###naive bayes (multinomial theoram)

In [45]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count,y_train)

MultinomialNB()

In [46]:
X_test_count = v.transform(X_test)
model.score(X_test_count, y_test)

0.9760247486465584

###pipeline

In [47]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [48]:
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [49]:
clf.score(X_test,y_test)

0.9760247486465584

In [50]:
#the first mail is not spam the second is spam
emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]
clf.predict(emails)

array([0, 1])